In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader, prep_config
from pure_funcs import denumpyize, numpyize, get_template_live_config, candidate_to_live_config, calc_spans, get_template_live_config, analyze_fills
from procedures import dump_live_config, load_live_config, add_argparse_args
from njit_funcs import calc_emas
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.optimize_config_path = 'configs/optimize/default.hjson'
        self.symbol = 'XMRUSDT'
        self.user = 'your_user_name'
        self.start_date = '2020-12-09'
        self.end_date = '2021-06-10'
config = await prep_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_data()
prices, buyer_maker, timestamps = data
print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
config['starting_balance'] = 100.0
config['latency_simulation_ms'] = 750
config['maker_fee'] = 0.00018

In [ ]:
template = get_template_live_config(n_spans=config['n_spans'])
#template = load_live_config('configs/live/binance_btsusdt.json')
template

In [ ]:
spans = calc_spans(template['min_span'], template['max_span'], template['n_spans'])
print(spans)
emas = calc_emas(prices, spans)
ratios = np.c_[prices, emas[:,:-1]] / emas
df = pd.DataFrame({**{'price': prices, 'buyer_maker': buyer_maker, 'timestamp': timestamps},
                   **{f'MA_{i}': emas[:,i] for i in range(len(emas[0]))},
                   **{f'ratio_{i}': ratios[:,i] for i in range(len(emas[0]))},
                      'MA_band_lower': emas.min(axis=1), 'MA_band_upper': emas.max(axis=1)})
df

In [ ]:
df[['MA_band_lower', 'MA_band_upper']].iloc[::100].plot()

In [ ]:
# tune it...

#template['shrt']['iprc_const'] = 1.0085
#template['long']['rprc_MAr_coeffs'][0][0] = -0.005

template['long']['rprc_PBr_coeffs'][0][1] = -0.14
template['shrt']['rprc_PBr_coeffs'][0][1] = 0.14
template['long']['rqty_const'] = 0.2
template['shrt']['rqty_const'] = 0.2

template['long']['iprc_const'] = 1.0
template['shrt']['iprc_const'] = 1.0
template['long']['iqty_const'] = 0.1
template['shrt']['iqty_const'] = 0.1
template['long']['leverage'] = 40.0
template['shrt']['leverage'] = 40.0
template['long']['rprc_const'] = 0.98
template['shrt']['rprc_const'] = 1.02
template['min_span'] = 9000.0
template['max_span'] = 360000.0
template['long']['markup_const'] = 1.012
template['shrt']['markup_const'] = 0.988
template['long']['rprc_MAr_coeffs'][2][1] = 0.0
template['shrt']['rprc_MAr_coeffs'][2][1] = 0.0
template['long']['rprc_MAr_coeffs'][2][0] = 0.0
template['shrt']['rprc_MAr_coeffs'][2][0] = -0.0








# etc

In [ ]:
config_to_test = {**config, **numpyize(template)}
dump_live_config(config_to_test, 'tmp/hand_tuned.json')

In [ ]:
sts = time()
fills, info = backtest(config_to_test, data)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
fdf, analysis = analyze_fills(fills, result, timestamps[0], timestamps[-1])
pprint.pprint(analysis)
fdf

In [ ]:
pprint.pprint(analysis)
pprint.pprint(template)

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
fdf.head(60)

In [ ]:
fdf[fdf.type.str.contains('long')].timestamp.diff().max() / (1000 * 60 * 60)

In [ ]:
fdf.timestamp.diff().sort_values() / (1000 * 60 * 60)

In [ ]:
plot_fills(df, fdf.loc[10410307:10410307+100000])

In [ ]:
sts = time()
fills, info = backtest(config_to_test, data)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
fdf, analysis = analyze_fills(fills, result, timestamps[0], timestamps[-1])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
# load external fills to inspect

fdf = pd.read_csv('backtests/binance/BTSUSDT/plots/2021-06-12T114030/fills.csv').set_index('trade_id')

In [ ]:
longs = fdf[fdf.type.str.contains('long')]
shrts = fdf[fdf.type.str.contains('shrt')]

In [ ]:
longs

In [ ]:
longs.tail(60)

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()